# API
- api documentation:- https://documenter.getpostman.com/view/9564387/TzRPip7u
- cowin api:- https://apisetu.gov.in/public/api/cowin/cowin-public-v2#/v2/appointment

In [1]:
# import the modules
import requests
import json
import time
from datetime import datetime, timedelta
from IPython.display import display, Audio
import pandas as pd
import numpy as np

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
}

base_url = 'https://cdn-api.co-vin.in/api/v2/admin/location/states'


response = requests.get(base_url, headers=headers)
states = response.json()

# kerala state code
for state in states['states']:
    if state['state_name'] == 'Kerala':
        print(state)
        state_of_int = state['state_id']
        
# get the district list
dist_url = 'https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}'.format(
    state_of_int
)

state_dists = requests.get(dist_url, headers=headers)

for dist in state_dists.json()['districts']:
    if dist['district_name'] == 'Ernakulam':
        print(dist)
        int_dist = dist['district_id']

{'state_id': 17, 'state_name': 'Kerala'}
{'district_id': 307, 'district_name': 'Ernakulam'}


In [2]:
# Functions
def hospital_details(hosptilas, seats=1):
    
    pre_time = []
    hospital = []
    address = []
    pincode = []
    date = []
    fee_type = []
    age = []
    vaccine = []
    doses = []
    
    table = pd.DataFrame()
    
    for center in hospitals.json()['centers']:
           
        for session in center['sessions']:
            if session['available_capacity_dose1'] >= seats:
                
                pre_time.append(datetime.now().strftime('%H:%M:%S'))
                hospital.append(center['name'])
                address.append(center['address'])
                pincode.append(center['pincode'])
                fee_type.append(center['fee_type'])
                date.append(session['date'])
                age.append(session['min_age_limit'])
                vaccine.append(session['vaccine'])
                doses.append(session['available_capacity_dose1'])
                
            
        table = pd.DataFrame({
            'Time': pre_time,
            'Hospital': hospital,
            'Address': address,
            'Pincode': np.array(pincode).astype(np.int),
            'Date': date,
            'Fee Type': fee_type,
            'Age': age,
            'Vaccine': vaccine,
            'No of Doses': doses
        })
            
    return table

def date_gen(days=6):

    date = [
        (datetime.today()+timedelta(i)).strftime('%d-%m-%Y')
        for i in range(days)
    ]
    
    return date

## Need to change the date inputs

In [3]:
# date input
DATES = date_gen(days=6)

# no of slots to show vaccant to show a hospital
SEATS = 5

# filter list based on specific req
FILTER = {
    
    'Vaccine': ['COVISHIELD'],
    'Fee Type': ['Free', 'Paid']
    
}

In [4]:
def table_filter(table, filter=FILTER):
    
    filtered_table = table
    
    if len(filter) != 0:
        for key, item in filter.items():
            fil = filtered_table[key].isin(item)
            filtered_table = filtered_table.loc[fil]
        
    return filtered_table

In [5]:
for _ in range(1000):
    table = []
    for date in DATES:
        calender_url = 'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}'.format(
            int_dist, date
        ) 
        
        hospitals = requests.get(calender_url, headers=headers)
        date_table = hospital_details(hospitals, seats=SEATS)
        table.append(date_table)
        
    table = pd.concat(table, axis=0, ignore_index=True)
    table.drop_duplicates(inplace=True)
    
    # write the slot to disk
    if len(table) > 0:
        table.to_csv('./cowin vaccine slots/table_{}.csv'.format(
            datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        )
        
    table = table_filter(table)
    
    if len(table) > 0:
        display(table)
        display(Audio(
            url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', 
            autoplay=True
        ))
        
    else:
        print('No slots are available')
        print(f"time: {datetime.now().strftime('%H:%M:%S')}")
        
        
    time.sleep(2*60)


No slots are available
time: 10:17:24
No slots are available
time: 10:19:26
No slots are available
time: 10:21:28
No slots are available
time: 10:23:29
No slots are available
time: 10:25:33
No slots are available
time: 10:27:35
No slots are available
time: 10:29:37
No slots are available
time: 10:31:38
No slots are available
time: 10:33:40
No slots are available
time: 10:35:42
No slots are available
time: 10:37:44
No slots are available
time: 10:39:47
No slots are available
time: 10:41:49
No slots are available
time: 10:43:51
No slots are available
time: 10:45:52
No slots are available
time: 10:47:54
No slots are available
time: 10:49:57
No slots are available
time: 10:52:01
No slots are available
time: 10:54:06
No slots are available
time: 10:56:09
No slots are available
time: 10:58:11
No slots are available
time: 11:00:13
No slots are available
time: 11:02:15
No slots are available
time: 11:04:17
No slots are available
time: 11:06:19
No slots are available
time: 11:08:21
No slots are

,Time,Hospital,Address,Pincode,Date,Fee Type,Age,Vaccine,No of Doses
0,12:31:45,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,45.0,COVISHIELD,145.0
1,12:31:45,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,18.0,COVISHIELD,305.0


,Time,Hospital,Address,Pincode,Date,Fee Type,Age,Vaccine,No of Doses
0,12:33:47,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,45.0,COVISHIELD,145.0
1,12:33:47,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,18.0,COVISHIELD,305.0


,Time,Hospital,Address,Pincode,Date,Fee Type,Age,Vaccine,No of Doses
0,12:35:49,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,45.0,COVISHIELD,145.0
1,12:35:49,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,18.0,COVISHIELD,305.0
2,12:35:49,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,45.0,COVISHIELD,138.0
3,12:35:49,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,18.0,COVISHIELD,238.0


,Time,Hospital,Address,Pincode,Date,Fee Type,Age,Vaccine,No of Doses
0,12:37:51,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,45.0,COVISHIELD,62.0
1,12:37:51,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,45.0,COVISHIELD,138.0
2,12:37:51,Renai Medicity Ernakulam,Near Palarivattom Metro Station Palarivattom P...,682025,13-07-2021,Paid,18.0,COVISHIELD,238.0


...

,Time,Hospital,Address,Pincode,Date,Fee Type,Age,Vaccine,No of Doses
0,15:15:07,Lakshmi Hospital Ernakulam,Lakshmi Hospital Divans Rd Pallimukku Kochi,682016,13-07-2021,Paid,18.0,COVISHIELD,137.0
1,15:15:07,GOWRI LAKSHMI MEDICAL CENTRE,Thiruvairanikulam Aluva Kerala,683580,13-07-2021,Paid,18.0,COVISHIELD,100.0
2,15:15:07,GOWRI LAKSHMI MEDICAL CENTRE,Thiruvairanikulam Aluva Kerala,683580,13-07-2021,Paid,45.0,COVISHIELD,25.0


,Time,Hospital,Address,Pincode,Date,Fee Type,Age,Vaccine,No of Doses
0,15:17:09,GOWRI LAKSHMI MEDICAL CENTRE,Thiruvairanikulam Aluva Kerala,683580,13-07-2021,Paid,18.0,COVISHIELD,41.0
1,15:17:09,GOWRI LAKSHMI MEDICAL CENTRE,Thiruvairanikulam Aluva Kerala,683580,13-07-2021,Paid,45.0,COVISHIELD,15.0


,Time,Hospital,Address,Pincode,Date,Fee Type,Age,Vaccine,No of Doses
0,15:19:11,GOWRI LAKSHMI MEDICAL CENTRE,Thiruvairanikulam Aluva Kerala,683580,13-07-2021,Paid,18.0,COVISHIELD,41.0
1,15:19:11,GOWRI LAKSHMI MEDICAL CENTRE,Thiruvairanikulam Aluva Kerala,683580,13-07-2021,Paid,45.0,COVISHIELD,15.0


No slots are available
time: 15:21:14
No slots are available
time: 15:23:16
No slots are available
time: 15:25:18
No slots are available
time: 15:27:20
No slots are available
time: 15:29:21
No slots are available
time: 15:31:23
No slots are available
time: 15:33:25
No slots are available
time: 15:35:28
No slots are available
time: 15:37:29
No slots are available
time: 15:39:31
No slots are available
time: 15:41:33
No slots are available
time: 15:43:35
No slots are available
time: 15:45:37
No slots are available
time: 15:47:39
No slots are available
time: 15:49:41
No slots are available
time: 15:51:42
No slots are available
time: 15:53:44
No slots are available
time: 15:55:46
No slots are available
time: 15:57:48
No slots are available
time: 15:59:50
No slots are available
time: 16:01:52
No slots are available
time: 16:03:54
No slots are available
time: 16:05:56
No slots are available
time: 16:07:58
No slots are available
time: 16:10:00
No slots are available
time: 16:12:02
No slots are

ConnectionError: HTTPSConnectionPool(host='cdn-api.co-vin.in', port=443): Max retries exceeded with url: /api/v2/appointment/sessions/public/calendarByDistrict?district_id=307&date=12-07-2021 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc048614a30>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

### mobile
9495621730

In [5]:
calender_url

'https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=307&date=08-07-2021'

In [44]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2021-07-04 16:50:54'